<a href="https://colab.research.google.com/github/wildanahkmwn/DBAlab_test/blob/main/DBAlab_test_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.optimizers import SGD
from torch.autograd import Variable
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import all dataset

df1 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject1.csv", header=None)
df2 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject2.csv", header=None)
df3 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject3.csv", header=None)
df4 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject4.csv", header=None)
df5 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject5.csv", header=None)
df6 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject6.csv", header=None)
df7 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject7.csv", header=None)
df8 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject8.csv", header=None)
df9 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject9.csv", header=None)
df10 = pd.read_csv("/content/drive/My Drive/DBALab-Test/Preprocessed/mHealth_subject10.csv", header=None)

In [4]:
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,-9.8184,0.009971,0.29563,0.004186,0.004186,0.63077,0.103900,-0.84053,-0.68762,-0.370000,-0.36327,0.299630,-8.6499,-4.57810,0.187760,-0.44902,-1.01030,0.034483,-2.35000,-1.610200,-0.030899,0
1,-9.8489,0.524040,0.37348,0.004186,0.016745,0.68389,0.085343,-0.83865,-0.68369,-0.197990,-0.18151,0.582980,-8.6275,-4.31980,0.023595,-0.44902,-1.01030,0.034483,-2.16320,-0.882540,0.326570,0
2,-9.6602,0.181850,0.43742,0.016745,0.037677,0.68113,0.085343,-0.83865,-0.68369,-0.374170,0.18723,0.438510,-8.5055,-4.27720,0.275720,-0.44902,-1.01030,0.034483,-1.61750,-0.165620,-0.030693,0
3,-9.6507,0.214220,0.24033,0.079540,0.117220,0.55031,0.085343,-0.83865,-0.68369,-0.017271,0.18366,0.575710,-8.6279,-4.31630,0.367520,-0.45686,-1.00820,0.025862,-1.07710,0.006945,-0.382620,0
4,-9.7030,0.303890,0.31156,0.221870,0.205130,0.71098,0.085343,-0.83865,-0.68369,-0.374390,-0.54671,0.445860,-8.7008,-4.14590,0.407290,-0.45686,-1.00820,0.025862,-0.53684,0.175900,-1.095500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144533,-9.4032,0.464170,-3.29510,0.272110,-0.050235,2.28640,-0.888680,-0.95497,0.88409,-3.039200,2.96620,-0.978980,-4.0618,4.61190,7.188900,-0.65490,0.54415,0.676720,40.18000,14.369000,-29.304000,0
144534,-9.5201,0.751360,-3.84820,-0.104660,-0.025118,2.24010,-0.903530,-0.96998,0.86640,-3.765300,5.35880,-0.844080,-4.4840,5.07440,6.441200,-0.65490,0.54415,0.676720,26.90100,-2.929500,-30.119000,0
144535,-9.3682,1.239600,-3.02530,-0.138150,-0.079540,2.39560,-0.903530,-0.96998,0.86640,-3.967500,5.72770,-0.122230,-3.1657,0.98528,7.528400,-0.65490,0.54415,0.676720,17.28800,5.330800,-30.393000,0
144536,-8.6702,1.185900,-3.01960,-0.066981,-0.050235,2.21890,-0.903530,-0.96998,0.86640,-4.690700,2.98270,-0.080312,-1.0351,-0.85876,6.123700,-0.65490,0.54415,0.676720,17.43000,19.502000,-28.727000,0


In [5]:
!pip install syft

     |████████████████████████████████| 440kB 2.8MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 4.0MB 11.3MB/s 
     |████████████████████████████████| 2.0MB 22.9MB/s 
     |████████████████████████████████| 9.0MB 41.5MB/s 
     |████████████████████████████████| 491kB 40.9MB/s 
     |████████████████████████████████| 2.2MB 45.7MB/s 
     |████████████████████████████████| 133kB 51.1MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 753.4MB 19kB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
     |████████████████████████████████| 1.8MB 39.7MB/s 
     |████████████████████████████████| 450kB 41.8MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |

In [6]:
#creating the virtual client
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

n_clients = 8
clients = []
for i in range(n_clients):
    client_name = 'client{}'.format(i)
    client = sy.VirtualWorker(hook, id = client_name)
    clients.append(client)
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [7]:
secure_worker

<VirtualWorker id:secure_worker #objects:0>

In [8]:
#concate all data to one dataset

df1['client_name'] = 1
df2['client_name'] = 2
df3['client_name'] = 3
df4['client_name'] = 4
df5['client_name'] = 5
df6['client_name'] = 6
df7['client_name'] = 7
df8['client_name'] = 8
df9['client_name'] = 9
df10['client_name'] = 10
df_all = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10])

#split to data train and test
df_train = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])
df_test = pd.concat([df9,df10])

In [9]:
#drop the 0 label
df_trainx = df_train[df_train[21]!=0]
nolabel_tx = df_train[df_train[21]==0]
df_testx = df_test[df_test[21]!=0]
nolabel_tsx = df_test[df_test[21]==0]

In [10]:
train_data = np.array(df_trainx, dtype = np.float32)
test_data = np.array(df_testx, dtype = np.float32)

In [11]:
def print_dataset(name, data):
    print('Dataset {}. Shape: {}'.format(name, data.shape))
    print(data)

In [12]:
print_dataset('Train', train_data)

Dataset Train. Shape: (147456, 23)
[[-9.7788e+00  5.5690e-01  1.1975e+00 ... -3.5020e-01  1.0000e+00
   1.0000e+00]
 [-9.7733e+00  2.7880e-01  7.3036e-01 ...  3.7363e-01  1.0000e+00
   1.0000e+00]
 [-9.8609e+00  1.1561e-01  7.9988e-01 ... -3.5020e-01  1.0000e+00
   1.0000e+00]
 ...
 [ 5.7627e+00  7.5603e-01 -3.1741e+00 ... -3.1726e+02  6.0000e+00
   8.0000e+00]
 [ 2.5956e+00  2.0397e+00  6.4673e-01 ... -2.9581e+02  6.0000e+00
   8.0000e+00]
 [-2.2642e+00  5.8287e-01  5.1051e+00 ... -2.2159e+02  6.0000e+00
   8.0000e+00]]


In [13]:
def get_input_and_output(data):
    input = Variable(torch.tensor(data[:, :-2], dtype = torch.float32))
    output1 = Variable(torch.tensor(data[:, -2], dtype = torch.float32))
    return input, output1

input, output1= get_input_and_output(train_data)
test_input, test_output1 = get_input_and_output(test_data)

to_percent = lambda x: '{:.2f}%'.format(x)

In [14]:
print_dataset('Train', output1)

Dataset Train. Shape: torch.Size([147456])
tensor([1., 1., 1.,  ..., 6., 6., 6.])


In [15]:
#create model in this case, use LogisticRegression
input_size = 21
learning_rate = 0.01
num_iterations = 20000

class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))


In [16]:
#send the data to each client
client_features = []
client_targets = []
for i in range(n_clients):
    train_data2 = df_trainx[df_trainx['client_name']==i+1]
    train_data2 = np.array(train_data2, dtype = np.float32)
    train_data3 = th.tensor(train_data2, dtype = torch.float32, requires_grad=True)
    features = train_data3[:, :-2].clone().detach().requires_grad_(True)
    targets = train_data3[:, -2][:, None].clone().detach() 
    client_features.append(features.send(clients[i]))
    client_targets.append(targets.send(clients[i]))
model = LogisticRegression()
print(model)

LogisticRegression(
  (linear): Linear(in_features=21, out_features=1, bias=True)
)


Define some functions to train the machine learning model in a federated way while keeping track of the training loss and the training accuracy.
The whole process is done in a trusted aggregator, in 100 iterations. (We can vary the number of iterations.). At each iteration, a copy of the shared model is sent to all the 8 client. Each client trains its own local model with its own local dataset, in 5 local iterations. (We can vary the number of local iterations.) Each local model improves a little bit in its own direction. Then we compute the local losses and local accuracies to keep track of them. So, we will able to create graphs of the learning curves: Training Losses versus Iterations and Training Accuracies versus Iterations. We send the local models to the trusted aggregator that will average all the model updates. This averaged model is the shared model that is sent to all the 8 client at the begining of each iteration.

In [17]:
iterations = 100 #2000 #can vary the number of iteration
worker_iterations = 5 #can vary the number of local iteration

#create function for grap
def plot_federated_graphs(losses, accuracies):
    for i in range(n_clients):
        plt.plot(losses[i], label=f'Client {i}')
    legend = plt.legend(loc='upper right', shadow=True)
    plt.title(f"Training Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Training Loss")
    plt.show()
    for i in range(n_clients):
        plt.plot(accuracies[i], label=f'Client {i}')
    legend = plt.legend(loc='lower right', shadow=True)
    plt.title(f"Training Accuracy")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy (Percent %)")
    plt.show()

#create function for accuracy    
def compute_federated_accuracy(model, input, output):
    prediction = model(input)
    n_samples = prediction.shape[0]
    s = 0.
    for i in range(n_samples):
        p = 1. if prediction[i] >= 0.5 else 0.
        e = 1. if p == output[i] else 0.
        s += e
    return 100. * s / n_samples

#create function for run federated learning
def federated_learning(client_features, client_targets, test_input, test_output):
    model = LogisticRegression()
    criterion = torch.nn.BCELoss(size_average=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
    losses = [[] for i in range(n_clients)]
    accuracies = [[] for i in range(n_clients)]
    for iteration in range(iterations):
        models = [model.copy().send(clients[i]) for i in range(n_clients)]
        optimizers = [torch.optim.SGD(params = models[i].parameters(), lr = learning_rate) for i in range(n_clients)]
        for worker_iteration in range(worker_iterations):
            last_losses = []
            for i in range(n_clients):
                optimizers[i].zero_grad()
                prediction = models[i](client_features[i])
                loss = criterion(prediction, client_targets[i])
                loss.backward()
                optimizers[i].step()
                loss = loss.get().data.item()
                last_losses.append(loss)
        for i in range(n_clients):
            losses[i].append(last_losses[i])
            train_acc = compute_federated_accuracy(models[i], client_features[i], client_targets[i])
            accuracies[i].append(train_acc)
            models[i].move(secure_worker)
        with th.no_grad():
            avg_weight = sum([models[i].linear.weight.data for i in range(n_clients)]) / n_clients
            model.linear.weight.set_(avg_weight.get())
            avg_bias = sum([models[i].linear.bias.data for i in range(n_clients)]) / n_clients
            model.linear.bias.set_(avg_bias.get())
        if iteration % 100 == 0:
            losses_str = ['{:.4f}'.format(losses[i][-1]) for i in range(n_clients)]
            accuracies_str = [to_percent(accuracies[i][-1]) for i in range(n_clients)]
            print('Iteration={}, losses={}, accuracies={}'.format(iteration, losses_str, accuracies_str))
    plot_federated_graphs(losses, accuracies)
    test_acc = compute_accuracy(model, test_input, test_output)
    print('\nTesting Accuracy = {}'.format(to_percent(test_acc)))
    return model

In [ ]:
model = federated_learning(client_features, client_targets, test_input, test_output1)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  return ret
/usr/local/lib/python3.6/dist-packages/syft/frameworks/torch/tensors/interpreters/native.py:156: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  to_return = self.native_grad


Iteration=0, losses=['-36.4964', '-106.3997', '-80.3530', '-63.5449', '-81.7682', '-103.0116', '-84.2979', '-62.8671'], accuracies=['16.67%', '16.67%', '16.67%', '16.67%', '16.67%', '16.67%', '16.67%', '16.67%']


https://github.com/ivishalanand/Federated-Learning-on-Hospital-Data/blob/master/second_revision.ipynb